# funkSVD

Here it is!

In [2]:
import pandas as pd
import pyreclab
import seaborn as sns
import numpy as np
import scipy.sparse as sparse
import matplotlib.pyplot as plt

%matplotlib inline
sns.set(style="whitegrid")

In [3]:
# WARNING: ONLY RUN ONCE OR DATA WILL BE ERASED 

# Instance datasets
train_file = pd.read_csv('dataset/train.csv', sep=',', header=0)
test_file = pd.read_csv('dataset/validation.csv', sep=',', header=0)
anime_file = pd.read_csv('dataset/anime.csv', sep=',', header=0)

train_file.to_csv("dataset/train.csv", index=False, header=False)  # Remove 1st row from CSV
test_file.to_csv("dataset/validation.csv", index=False, header=False)
anime_file.to_csv("dataset/anime.csv", index=False, header=False)

train_file = pd.read_csv('dataset/train.csv', names = ['user_id','item_id','rating'] ,sep=',', header=0)
test_file = pd.read_csv('dataset/validation.csv', names = ['user_id','item_id','rating'], sep=',', header=0)
anime_file = pd.read_csv('dataset/anime.csv', names = ['anime_id','name','genre','type','episodes','rating','members'], sep=',', header=0)

train_file.head()

,user_id,item_id,rating
0,18266,51,-1
1,31006,8795,7
2,68084,14837,8
3,20881,1536,9
4,25996,1241,8


In [6]:
anime_file.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12292 entries, 0 to 12291
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   anime_id  12292 non-null  int64  
 1   name      12292 non-null  object 
 2   genre     12230 non-null  object 
 3   type      12267 non-null  object 
 4   episodes  12292 non-null  object 
 5   rating    12062 non-null  float64
 6   members   12292 non-null  int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 672.3+ KB


In [4]:
# definition and training of the svd object
svd = pyreclab.SVD(dataset='dataset/train.csv',
                   dlmchar=b',',
                   header=False,
                   usercol=0,
                   itemcol=1,
                   ratingcol=2)

# arbitrary first parameters :)
svd.train(factors=100, maxiter=100, lr=0.01, lamb=0.1)


In [5]:
# Evaluation of the model using the validation dataset
predlist, mae, rmse = svd.test(input_file='dataset/validation.csv',
                               dlmchar=b',',
                               header=False,
                               usercol=0,
                               itemcol=1,
                               ratingcol=2)

print(f"MAE: {mae}")
print(f"RMSE: {rmse}")


MAE: 2.5020502059635663
RMSE: 3.2543449783776808


In [9]:
# Recommendation for a arbitrary user
user_id = 99  # This can be changed
top_n = 10

recommendations = [int(item) for item in svd.recommend(str(user_id), top_n, includeRated=False)]

# Show titles of the recommended animes
anime_titles = anime_file.set_index("anime_id").loc[recommendations]["name"]
print(anime_titles)


anime_id
18195                             Little Busters!: Refrain
2594                                         Piano no Mori
21855                                       Hanamonogatari
16894                          Kuroko no Basket 2nd Season
25835                                            Shirobako
4477                           Nodame Cantabile: Paris-hen
2951     Gintama: Nanigoto mo Saiyo ga Kanjin nano de T...
93                                 Mobile Suit Gundam Seed
175                                      Tokyo Underground
5682                      Phantom: Requiem for the Phantom
Name: name, dtype: object


In [10]:
recommendList, maprec, ndcg = svd.testrec(input_file='dataset/validation.csv',
                                          dlmchar=b',',
                                          header=False,
                                          usercol=0,
                                          itemcol=1,
                                          ratingcol=2,
                                          topn=10,
                                          relevance_threshold=2,
                                          includeRated=False)

print(f"MAP@10: {maprec}")
print(f"NDCG@10: {ndcg}")


MAP@10: 0.0006283883789362374
NDCG@10: 0.001478087359544408
